# Does any transcript levels correlate with FBA-predicted metabolic fluxes?

## Introduction

Transcriptomics dataset collected from multiple scenarios and published in [Kannan et al 2019](https://doi.org/10.1093/insilicoplants/diz008) was provided from UoI-UC. FBA was used to predict metabolic fluxes under different scenarios and then transcript levels are compared to flux.

## Processing transcript data

In [1]:
import pandas as pd
df = pd.read_csv("/home/sanu/Downloads/gmax_leaf_CO2_normExprn_1.txt",sep="\t")

/home/sanu/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (0,2,3,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
len(df.columns) -17

229

### Seperating out different studies

Data on light intensity (PPFD) during plant growth was avialable for 5 (of 8) studies. When \[CO2\] is not avilable, ambient levels were assumed. Where day-length is not mentioned, as assumption that daylength is 12h was assumed. In the case of studies involving biotic and abiotic treatments (other than high \[CO2\] treatments), only the WT/control data was used in this analysis.

In [3]:
amb_list = list(df.columns[0:17])
ele_list = list(df.columns[0:17])
for x in df.columns:
    if "Amb" in x:
        amb_list.append(x)
    elif "Ele" in x:
        ele_list.append(x)
    else:
        continue

budRemove_list = list(df.columns[0:17])+["GSM569808.CEL","GSM569809.CEL","GSM569810.CEL",
                                         "GSM569811.CEL","GSM569812.CEL","GSM569813.CEL",
                                         "GSM569814.CEL"]
heatShock_list = list(df.columns[0:17])+["GSM643115.CEL","GSM643116.CEL","GSM643117.CEL","GSM643118.CEL"]
mycInfect_list = list(df.columns[0:17])+["GSM1088939_Schaar_19_wt_c1_S.CEL",
                                         "GSM1088940_Schaar_20_wt_c2_S.CEL",
                                         "GSM1088954_Schaar_25_nts_c1_S.CEL",
                                         "GSM1088955_Schaar_26_nts_c2_S.CEL",
                                         "GSM1088956_Schaar_27_nts_c3_S.CEL"]

In [4]:
df_amb = df[amb_list]
df_ele = df[ele_list]
df_budRemove = df[budRemove_list]
df_heatShock = df[heatShock_list]
df_mycInfect = df[mycInfect_list]

## Set up leaf model and add gene-protein-reaction (GPR) associations from PlantCyc

In [5]:
from cobra import io
from cobra.core import Metabolite
from cobra import flux_analysis
from IPython import display
import logging
logging.basicConfig()
logger = logging.getLogger('logger')

#import sbml file
fname = "/home/sanu/Documents/Scripts/git/plantcoremetabolism-model/PlantCoreMetabolism_v1_3_0.xml"
model = io.sbml.read_sbml_model(fname)
display.clear_output()

from sweetlovegroup.transform import fixModelCompatibilityIssueCobra015
model = fixModelCompatibilityIssueCobra015(model,fname)


In [6]:
len(model.reactions)

868

In [7]:
len(model.metabolites)

829

In [8]:
import pythoncyc
from Functions import *

soy = pythoncyc.select_organism("soy")


In [9]:
model = addGPR2Models(model,soy)

--------------
This list of metabolic reactions are ignored
['RXN_9650_p', '2_KETO_ADIPATE_DEHYDROG_RXN_m', 'Phytol_biosynthesis_p', 'CYSTEINE_AMINOTRANSFERASE_RXN_m', 'GLYCINE_TRNA_LIGASE_RXN_c', 'RXN66_1_c', 'RXN_9648_p', 'RXN-9651', 'Plastidial_ATP_Synthase_p', 'GGPP_biosynthesis_p', 'RXN_9653_p', 'lycopene_biosynthesis_p', 'RXN_2141_p', 'SUCCINYL_COA_HYDROLASE_RXN_m', 'PROTON_ATPase_c', 'MDA_Fd_Ascorbate_p', 'MercaptoPyruvateSulfurtransferase_m', 'Phytol_degradation_p', 'RXN_9652_p', 'A_B_oxidation_x', 'unlProtHYPO_c', 'Mitochondrial_ATP_Synthase_m', 'IPP_biosynthesis_c', 'Mehler_Reaction_p', 'Beta_Oxidation_x', 'HMBPP_synthesis_p', 'OROTATE_REDUCTASE_NADH_RXN_p', 'Ferredoxin_Plastoquinone_Reductase_p', 'RXN_9651_p', 'NADPH_Dehydrogenase_p', 'Plastoquinol_Oxidase_p', 'SUCCINATE_COA_LIGASE_GDP_FORMING_RXN_m', 'RXN_1781_v', 'PREPHENATE_DEHYDROGENASE_NADP_RXN_p', 'PREPHENATEDEHYDROG_RXN_p', 'MALEYLACETOACETATE_ISOMERASE_RXN_c', 'RXN_9654_p', 'LCYSDESULF_RXN_c', 'RXN_9958_NAD_m', 'HEXO

## Modelling fluxes under different scenarios

Create leaf model

In [10]:
########################################################
#This function was used to set up a C3 leaf diel model #
########################################################
def setupC3DielModel(core_model,transferMets,starch_sucrose_ratio=90.0/100):
    from cobra.core import Metabolite, Reaction
    import re

    #create two copies of model elements for day and night
    cobra_model2 = core_model.copy()
    for met in cobra_model2.metabolites:
        met.id = met.id+"1"
        met.compartment = met.compartment+"1"
    for rxn in cobra_model2.reactions:
        rxn.id = rxn.id+"1"

    cobra_model3 = core_model.copy()
    for met in cobra_model3.metabolites:
        met.id = met.id+"2"
        met.compartment = met.compartment+"2"
    for rxn in cobra_model3.reactions:
        rxn.id = rxn.id+"2"

    #merge the day and night model
    cobra_model = cobra_model2+cobra_model3
    for met in cobra_model3.metabolites:
        if not cobra_model.metabolites.__contains__(met.id):
            cobra_model.add_metabolites(met.copy())

    met1 = Metabolite("X_Phloem_contribution_t1",name="Phloem output during the day")
    cobra_model.reactions.get_by_id("Phloem_output_tx1").add_metabolites({met1:1})
    met2 = Metabolite("X_Phloem_contribution_t2",name="Phloem output during at night")
    cobra_model.reactions.get_by_id("Phloem_output_tx2").add_metabolites({met2:1})

    rxn = Reaction("diel_biomass")
    rxn.add_metabolites({met1:-3,met2:-1})
    rxn.lower_bound = 0
    rxn.upper_bound = 1000
    cobra_model.add_reaction(rxn)

    #Adding reactions to allow for day-night metabolite accumulations
    tmfile = open(transferMets,"r")
    tmset=set()
    for line in tmfile:
        tmset.add(line.replace("\n",""))

    for met in tmset:
        if met == "AMMONIUM_v" or met=="FRUCTAN_v":
            continue
        tempRxn = Reaction(met+"_dielTransfer")
        tempRxn.add_metabolites({cobra_model.metabolites.get_by_id(met+"1"):-1,cobra_model.metabolites.get_by_id(met+"2"):1})
        tempRxn.lower_bound=-1000
        if not ((met == "STARCH_p") or (met == "SUCROSE_v") or (met == "MAL_v") or (met == "aMAL_v") or (met == "NITRATE_v") or (met == "CIT_v") or (met == "aCIT_v") or (met == "PROTON_v")):
            tempRxn.lower_bound=0
        tempRxn.upper_bound=1000
        cobra_model.add_reaction(tempRxn)

    fractionMets=dict()
    for rxn in cobra_model.reactions:
        for met in rxn.metabolites.keys():
            prefix=""
            a=re.search("^a{1,3}",met.id)
            anion=""
            if a:
                anion=a.group(0)
                prefix=anion
            b=re.search("^b{1,3}",met.id)
            basic=""
            if b:
                basic=b.group(0)
                prefix=basic
            if ((not prefix == "") and met.compartment == "v1"):
                fractionMets[met]=prefix

    temp=cobra_model.copy()
    for met in fractionMets.keys():
        for rxn in met.reactions:
            if rxn.id.__contains__("_dielTransfer"):
                continue
            else:
                mainMet = met.id[len(fractionMets[met]):]
                coeff1 = temp.reactions.get_by_id(rxn.id).metabolites.get(temp.metabolites.get_by_id(mainMet))
                coeff2 = temp.reactions.get_by_id(rxn.id).metabolites.get(temp.metabolites.get_by_id(met.id))
                if not coeff1:
                    coeff1=0
                if not coeff2:
                    coeff2=0
                total = coeff1 + coeff2
                coeff1 = float(coeff1)/total
                coeff2 = float(coeff2)/total
                if cobra_model.reactions.has_id(met.id[0:len(met.id)-1]+"_dielTransfer"):
                    ub = temp.reactions.get_by_id(met.id[0:len(met.id)-1]+"_dielTransfer").upper_bound
                    lb = temp.reactions.get_by_id(met.id[0:len(met.id)-1]+"_dielTransfer").lower_bound
                    temp.reactions.get_by_id(met.id[0:len(met.id)-1]+"_dielTransfer").remove_from_model()
                    temp.reactions.get_by_id(mainMet[0:len(mainMet)-1]+"_dielTransfer").remove_from_model()
                    Reac = Reaction(mainMet[0:len(mainMet)-1]+"_dielTransfer",name=mainMet+"_dielTransfer")
                    Reac.add_metabolites({temp.metabolites.get_by_id(met.id[0:len(met.id)-1]+"1"):-coeff2,temp.metabolites.get_by_id(met.id[0:len(met.id)-1]+"2"):coeff2,temp.metabolites.get_by_id(mainMet[0:len(mainMet)-1]+"1"):-coeff1,temp.metabolites.get_by_id(mainMet[0:len(mainMet)-1]+"2"):coeff1})
                    Reac.lower_bound=lb
                    Reac.upper_bound=ub
                    temp.add_reaction(Reac)
                    print Reac.reaction
                break
    ####ADD CONSTRAINTS TO MODEL####
    cobra_model = temp.copy()

    #objective function
    cobra_model.reactions.get_by_id("diel_biomass").objective_coefficient=1
    #Leaves - light
    cobra_model.reactions.get_by_id("Sucrose_tx1").lower_bound=0
    cobra_model.reactions.get_by_id("Sucrose_tx1").upper_bound=0
    cobra_model.reactions.get_by_id("GLC_tx1").lower_bound=0
    cobra_model.reactions.get_by_id("GLC_tx1").upper_bound=0
    cobra_model.reactions.get_by_id("CO2_tx1").lower_bound=0
    cobra_model.reactions.get_by_id("NH4_tx1").lower_bound=0
    cobra_model.reactions.get_by_id("NH4_tx1").upper_bound=0
    #Leaves - dark
    cobra_model.reactions.get_by_id("Sucrose_tx2").lower_bound=0
    cobra_model.reactions.get_by_id("Sucrose_tx2").upper_bound=0
    cobra_model.reactions.get_by_id("GLC_tx2").lower_bound=0
    cobra_model.reactions.get_by_id("GLC_tx2").upper_bound=0
    cobra_model.reactions.get_by_id("Photon_tx2").lower_bound=0
    cobra_model.reactions.get_by_id("Photon_tx2").upper_bound=0
    cobra_model.reactions.get_by_id("NH4_tx2").lower_bound=0
    cobra_model.reactions.get_by_id("NH4_tx2").upper_bound=0
    cobra_model.reactions.get_by_id("CO2_tx2").upper_bound=0

    #Set pG6P transporter to 0
    cobra_model.reactions.get_by_id("G6P_Pi_pc1").lower_bound=0
    cobra_model.reactions.get_by_id("G6P_Pi_pc1").upper_bound=0
    cobra_model.reactions.get_by_id("G6P_Pi_pc2").lower_bound=0
    cobra_model.reactions.get_by_id("G6P_Pi_pc2").upper_bound=0

    #Turn off PTOX
    cobra_model.reactions.get_by_id("Plastoquinol_Oxidase_p1").lower_bound=0
    cobra_model.reactions.get_by_id("Plastoquinol_Oxidase_p1").upper_bound=0

    #nitrate uptake constrain
    Nitrate_balance = Metabolite("Nitrate_bal_c", name = "Weights to balance nitrate uptake", compartment = "c1")
    cobra_model.reactions.get_by_id("Nitrate_ec1").add_metabolites({Nitrate_balance:-2})
    cobra_model.reactions.get_by_id("Nitrate_ec2").add_metabolites({Nitrate_balance:3})

    #Rubisco balance
    Rubisco_balance = Metabolite("rubisco_bal_p1", name = "Weights to balance RuBP carboxygenase oxygenase balance", compartment = "p1")
    cobra_model.reactions.get_by_id("RXN_961_p1").add_metabolites({Rubisco_balance:3})
    cobra_model.reactions.get_by_id("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p1").add_metabolites({Rubisco_balance:-1})

    #generic ATPase and NADPH oxidase
    Maintenance_constraint = Metabolite("ATPase_NADPHoxidase_constraint_c1",name =  "ATPase_NADPHoxidase_constraint_c1", compartment = "c1")
    Maintenance_constraint2 = Metabolite("ATPase_NADPHoxidase_constraint_c2",name =  "ATPase_NADPHoxidase_constraint_c2", compartment = "c2")
    Maintenance_constraint3 = Metabolite("Light_dark_maintainence_constraint",name =  "Light_dark_maintainence_constraint", compartment = "c1")
    cobra_model.reactions.get_by_id("ATPase_tx1").add_metabolites({Maintenance_constraint:1,Maintenance_constraint3:1})
    cobra_model.reactions.get_by_id("ATPase_tx2").add_metabolites({Maintenance_constraint2:1,Maintenance_constraint3:-1})
    cobra_model.reactions.get_by_id("NADPHoxc_tx1").add_metabolites({Maintenance_constraint:-3})
    cobra_model.reactions.get_by_id("NADPHoxc_tx2").add_metabolites({Maintenance_constraint2:-3})
    cobra_model.reactions.get_by_id("NADPHoxm_tx1").add_metabolites({Maintenance_constraint:-3})
    cobra_model.reactions.get_by_id("NADPHoxm_tx2").add_metabolites({Maintenance_constraint2:-3})
    cobra_model.reactions.get_by_id("NADPHoxp_tx1").add_metabolites({Maintenance_constraint:-3})
    cobra_model.reactions.get_by_id("NADPHoxp_tx2").add_metabolites({Maintenance_constraint2:-3})

    ##constrain sucrose and starch storage
    Sucorse_starch_balance = Metabolite("sucrose_starch_bal_c", name = "Weights to balance sucrose-starch uptake", compartment = "c1")
    cobra_model.reactions.get_by_id("SUCROSE_v_dielTransfer").add_metabolites({Sucorse_starch_balance:-1*starch_sucrose_ratio})
    cobra_model.reactions.get_by_id("STARCH_p_dielTransfer").add_metabolites({Sucorse_starch_balance:1})

    #Plastid enolase was not detected in Arabidopsis mesophyll tissue
    cobra_model.reactions.get_by_id("2PGADEHYDRAT_RXN_p1").lower_bound=0
    cobra_model.reactions.get_by_id("2PGADEHYDRAT_RXN_p1").upper_bound=0
    cobra_model.reactions.get_by_id("2PGADEHYDRAT_RXN_p2").lower_bound=0
    cobra_model.reactions.get_by_id("2PGADEHYDRAT_RXN_p2").upper_bound=0

    #Setting chloroplastic NADPH dehydrogenase to 0  ((Yamamoto et al., 2011)
    cobra_model.reactions.get_by_id("NADPH_Dehydrogenase_p1").lower_bound=0
    cobra_model.reactions.get_by_id("NADPH_Dehydrogenase_p1").upper_bound=0
    cobra_model.reactions.get_by_id("NADPH_Dehydrogenase_p2").lower_bound=0
    cobra_model.reactions.get_by_id("NADPH_Dehydrogenase_p2").upper_bound=0

    #ATP_ADP_Pi constrained to 0 because while there is evidence for its existance, it does not carry flux during the day
    cobra_model.reactions.get_by_id("ATP_ADP_Pi_pc1").lower_bound = 0
    cobra_model.reactions.get_by_id("ATP_ADP_Pi_pc1").upper_bound = 0

    return cobra_model


In [11]:
leaf_model = model.copy()

from sweetlovegroup import transform
leaf_model = setupC3DielModel(leaf_model,"MetabolitesToTransfer.txt",starch_sucrose_ratio=5)

for rxn in leaf_model.reactions:
    if rxn.lower_bound==-1000:
        rxn.lower_bound=-2000
    if rxn.upper_bound==1000:
        rxn.upper_bound=2000

leaf_model.reactions.diel_biomass.objective_coefficient=1
sol = flux_analysis.parsimonious.pfba(leaf_model)

0.5 CIT_v1 + 0.5 aCIT_v1 <=> 0.5 CIT_v2 + 0.5 aCIT_v2
0.7 MAL_v1 + 0.3 aMAL_v1 <=> 0.7 MAL_v2 + 0.3 aMAL_v2
bHIS_v1 --> bHIS_v2


### Scenario 1

PPFD = 1500  
\[CO2\] = 400  
net CO2 uptake rate = 21.645569620253173 ([Kannan et al 2019](https://doi.org/10.1093/insilicoplants/diz008))

In [34]:
PPFD = 1500
ATPase = (0.0049*PPFD) + 2.7851

from sweetlovegroup import analysis
temp = leaf_model.copy()
temp.reactions.get_by_id("Photon_tx1").lower_bound = 0
temp.reactions.get_by_id("Photon_tx1").upper_bound = PPFD
temp.reactions.get_by_id("ATPase_tx1").lower_bound = 4
temp.reactions.get_by_id("ATPase_tx1").upper_bound = 4
out_400 = analysis.estimateOutputFromNetCO2(temp,21.645569620253173)

leaf_model_400 = leaf_model.copy()
leaf_model_400.reactions.get_by_id("Photon_tx1").lower_bound = 0
leaf_model_400.reactions.get_by_id("Photon_tx1").upper_bound = PPFD
leaf_model_400.reactions.get_by_id("ATPase_tx1").lower_bound = 4
leaf_model_400.reactions.get_by_id("ATPase_tx1").upper_bound = 4
leaf_model_400.reactions.diel_biomass.lower_bound = out_400
leaf_model_400.reactions.diel_biomass.upper_bound = out_400
sol = flux_analysis.parsimonious.pfba(leaf_model_400)
print("Phloem output = ")
print(leaf_model_400.reactions.diel_biomass.flux)


totalTranscript_400 = dict()
for rxn in leaf_model.reactions:
    x=0
    for gene in rxn.genes:
        #print gene
        y = df_amb[df_amb["Glyma2.0"]==gene.id][df_amb.columns[17:]].mean().mean()
        #print y
        if pd.isna(y):
            x = x + 0
        else:
            x = x + y
    totalTranscript_400[rxn.id]=x

1500
4
21.6455696203
Phloem output = 
0.499446983254


### Scenario 2

PPFD = 1500  
\[CO2\] = 550  
net CO2 uptake rate = 24.43037974683546 ([Kannan et al 2019](https://doi.org/10.1093/insilicoplants/diz008))

In [13]:
PPFD = 1500
ATPase = (0.0049*PPFD) + 2.7851

from sweetlovegroup import analysis
temp = leaf_model.copy()
temp.reactions.get_by_id("Photon_tx1").lower_bound = 0
temp.reactions.get_by_id("Photon_tx1").upper_bound = PPFD
temp.reactions.get_by_id("ATPase_tx1").lower_bound = 4
temp.reactions.get_by_id("ATPase_tx1").upper_bound = 4
out_550 = analysis.estimateOutputFromNetCO2(temp,24.43037974683546)

leaf_model_550 = leaf_model.copy()
leaf_model_550.reactions.get_by_id("Photon_tx1").lower_bound = 0
leaf_model_550.reactions.get_by_id("Photon_tx1").upper_bound = PPFD
leaf_model_550.reactions.get_by_id("ATPase_tx1").lower_bound = 4
leaf_model_550.reactions.get_by_id("ATPase_tx1").upper_bound = 4
leaf_model_550.reactions.diel_biomass.lower_bound = out_550
leaf_model_550.reactions.diel_biomass.upper_bound = out_550
sol = flux_analysis.parsimonious.pfba(leaf_model_550)
print("Phloem output = ")
print(leaf_model_550.reactions.diel_biomass.flux)


totalTranscript_550 = dict()
for rxn in leaf_model.reactions:
    x=0
    for gene in rxn.genes:
        #print gene
        y = df_ele[df_ele["Glyma2.0"]==gene.id][df_ele.columns[17:]].mean().mean()
        #print y
        if pd.isna(y):
            x = x + 0
        else:
            x = x + y
    totalTranscript_550[rxn.id]=x

1500
4
24.4303797468
Phloem output = 
0.567924348284


### Scenario 3

PPFD = 400  
\[CO2\] = 400  
net CO2 uptake rate = 21.645569620253173 ([Kannan et al 2019](https://doi.org/10.1093/insilicoplants/diz008))

In [14]:
PPFD = 400
ATPase = (0.0049*PPFD) + 2.7851

from sweetlovegroup import analysis
temp = leaf_model.copy()
temp.reactions.get_by_id("Photon_tx1").lower_bound = 0
temp.reactions.get_by_id("Photon_tx1").upper_bound = PPFD
temp.reactions.get_by_id("ATPase_tx1").lower_bound = 4
temp.reactions.get_by_id("ATPase_tx1").upper_bound = 4
out_400 = analysis.estimateOutputFromNetCO2(temp,21.645569620253173)

leaf_model_budRemove = leaf_model.copy()
leaf_model_budRemove.reactions.get_by_id("Photon_tx1").lower_bound = 0
leaf_model_budRemove.reactions.get_by_id("Photon_tx1").upper_bound = PPFD
leaf_model_budRemove.reactions.get_by_id("ATPase_tx1").lower_bound = 4
leaf_model_budRemove.reactions.get_by_id("ATPase_tx1").upper_bound = 4
leaf_model_budRemove.reactions.diel_biomass.lower_bound = out_400
leaf_model_budRemove.reactions.diel_biomass.upper_bound = out_400
sol = flux_analysis.parsimonious.pfba(leaf_model_budRemove)
print("Phloem output = ")
print(leaf_model_budRemove.reactions.diel_biomass.flux)


totalTranscript_budRemove = dict()
for rxn in leaf_model.reactions:
    x=0
    for gene in rxn.genes:
        #print gene
        y = df_budRemove[df_budRemove["Glyma2.0"]==gene.id][df_budRemove.columns[17:]].mean().mean()
        #print y
        if pd.isna(y):
            x = x + 0
        else:
            x = x + y
    totalTranscript_budRemove[rxn.id]=x

400
4
21.6455696203
Phloem output = 
0.499446983252


### Scenario 4

PPFD = 350  
\[CO2\] = 380  
net CO2 uptake rate = 12 (\[CO2\]=370) ([Haile nd Higley 2003](https://doi.org/10.1603/0046-225x-32.3.433))

In [15]:
PPFD = 400
ATPase = (0.0049*PPFD) + 2.7851

from sweetlovegroup import analysis
temp = leaf_model.copy()
temp.reactions.get_by_id("Photon_tx1").lower_bound = 0
temp.reactions.get_by_id("Photon_tx1").upper_bound = PPFD
temp.reactions.get_by_id("ATPase_tx1").lower_bound = 4
temp.reactions.get_by_id("ATPase_tx1").upper_bound = 4
out_370 = analysis.estimateOutputFromNetCO2(temp,12)

leaf_model_heatShock = leaf_model.copy()
leaf_model_heatShock.reactions.get_by_id("Photon_tx1").lower_bound = 0
leaf_model_heatShock.reactions.get_by_id("Photon_tx1").upper_bound = PPFD
leaf_model_heatShock.reactions.get_by_id("ATPase_tx1").lower_bound = 4
leaf_model_heatShock.reactions.get_by_id("ATPase_tx1").upper_bound = 4
leaf_model_heatShock.reactions.diel_biomass.lower_bound = out_370
leaf_model_heatShock.reactions.diel_biomass.upper_bound = out_370
sol = flux_analysis.parsimonious.pfba(leaf_model_heatShock)
print("Phloem output = ")
print(leaf_model_heatShock.reactions.diel_biomass.flux)


totalTranscript_heatShock = dict()
for rxn in leaf_model.reactions:
    x=0
    for gene in rxn.genes:
        #print gene
        y = df_heatShock[df_heatShock["Glyma2.0"]==gene.id][df_heatShock.columns[17:]].mean().mean()
        #print y
        if pd.isna(y):
            x = x + 0
        else:
            x = x + y
    totalTranscript_heatShock[rxn.id]=x

400
4
12
Phloem output = 
0.262112136056


### Scenario 5

PPFD = 200  
\[CO2\] = ? , assuming 390 based on publication date and CO2 data from the year (https://www.co2.earth/monthly-co2)  
net CO2 uptake rate = 4.693877551020307 (\[CO2\]=390 and assuming Ci = 0.72\*390 = 280.8) ([Bunce et al 2019](https://doi.org/10.1093/insilicoplants/diz008))

In [16]:
PPFD = 200
ATPase = (0.0049*PPFD) + 2.7851

from sweetlovegroup import analysis
temp = leaf_model.copy()
temp.reactions.get_by_id("Photon_tx1").lower_bound = 0
temp.reactions.get_by_id("Photon_tx1").upper_bound = PPFD
temp.reactions.get_by_id("ATPase_tx1").lower_bound = 4
temp.reactions.get_by_id("ATPase_tx1").upper_bound = 4
out_370 = analysis.estimateOutputFromNetCO2(temp,4.693877551020307)

leaf_model_mycInfect = leaf_model.copy()
leaf_model_mycInfect.reactions.get_by_id("Photon_tx1").lower_bound = 0
leaf_model_mycInfect.reactions.get_by_id("Photon_tx1").upper_bound = PPFD
leaf_model_mycInfect.reactions.get_by_id("ATPase_tx1").lower_bound = 4
leaf_model_mycInfect.reactions.get_by_id("ATPase_tx1").upper_bound = 4
leaf_model_mycInfect.reactions.diel_biomass.lower_bound = out_370
leaf_model_mycInfect.reactions.diel_biomass.upper_bound = out_370
sol = flux_analysis.parsimonious.pfba(leaf_model_mycInfect)
print("Phloem output = ")
print(leaf_model_mycInfect.reactions.diel_biomass.flux)


totalTranscript_mycInfect = dict()
for rxn in leaf_model.reactions:
    x=0
    for gene in rxn.genes:
        #print gene
        y = df_mycInfect[df_mycInfect["Glyma2.0"]==gene.id][df_mycInfect.columns[17:]].mean().mean()
        #print y
        if pd.isna(y):
            x = x + 0
        else:
            x = x + y
    totalTranscript_mycInfect[rxn.id]=x

200
4
4.69387755102
Phloem output = 
0.084150928083


In [17]:
modelDict = dict()
modelDict[1] = leaf_model_400
modelDict[2] = leaf_model_550
modelDict[3] = leaf_model_budRemove
modelDict[4] = leaf_model_heatShock
modelDict[5] = leaf_model_mycInfect

totalTranscriptDict = dict()
totalTranscriptDict[1] = totalTranscript_400
totalTranscriptDict[2] = totalTranscript_550
totalTranscriptDict[3] = totalTranscript_budRemove
totalTranscriptDict[4] = totalTranscript_heatShock
totalTranscriptDict[5] = totalTranscript_mycInfect

In [18]:
import numpy as np
df_ratiosDict = dict()
for i in range(1,6):
    for j in range(i+1,6):
        print(str(i)+"-"+str(j))
        rxnList = list()
        transcriptList = list()
        fluxList = list()
        x=1
        for rxnID in totalTranscriptDict[i].keys():
            if rxnID[len(rxnID)-1]=="2":
                continue
            rxnList.append(rxnID)
            a=0
            b=0
            if round(totalTranscriptDict[j][rxnID],3)==0 and round(totalTranscriptDict[i][rxnID],3)==0:
                transcriptList.append(0)
            elif round(totalTranscriptDict[j][rxnID],3)==0:
                transcriptList.append(np.nan)
            else:
                a = totalTranscriptDict[i][rxnID]/totalTranscriptDict[j][rxnID]
                transcriptList.append(a)
            if round(modelDict[i].reactions.get_by_id(rxnID).flux,3)==0 and round(modelDict[j].reactions.get_by_id(rxnID).flux,3)==0:
                fluxList.append(0)
            elif round(modelDict[j].reactions.get_by_id(rxnID).flux,3)==0:
                fluxList.append(np.nan)
            else:
                b = modelDict[i].reactions.get_by_id(rxnID).flux/modelDict[j].reactions.get_by_id(rxnID).flux
                fluxList.append(b)
            if not (a==0 and b==0):
                if b == 0:
                    continue
                if round(a/b,2)>0.98 and round(a/b,2)<1.02:
                    print rxnID
                    print a/b
                    print x
                    x=x+1
        
        print("--------------")
        df_ratios = pd.DataFrame(data={"rxn ID":rxnList,"Transcript ratio":transcriptList,"Flux ratio":fluxList})
        df_ratiosDict[str(i)+"-"+str(j)]=df_ratios
    

1-2
INORGPYROPHOSPHAT_RXN_p1
1.0071318115906047
1
--------------
1-3
ALANINE_AMINOTRANSFERASE_RXN_c1
0.9944036769729111
1
SHIKIMATE_KINASE_RXN_p1
1.001633882847672
2
GLUTAMINESYN_RXN_c1
0.9865517598494091
3
GLUTAMINESYN_RXN_m1
0.9865517598499515
4
GPH_RXN_p1
1.0126859397579502
5
PHOSPHORIBULOKINASE_RXN_p1
0.9957709147659001
6
BRANCHED_CHAINAMINOTRANSFERLEU_RXN_p1
1.0096343985074867
7
ATPPHOSPHORIBOSYLTRANS_RXN_p1
0.9874198939265711
8
TRIOSEPISOMERIZATION_RXN_p1
1.0006040038817492
9
2PGADEHYDRAT_RXN_c1
1.0072084092830293
10
MALATE_DEHYDROGENASE_NADP_RXN_p1
1.0117130616584715
11
--------------
1-4
--------------
1-5
--------------
2-3
SUCROSE_PHOSPHATASE_RXN_c1
0.9901831589752683
1
GLUTAMATE_N_ACETYLTRANSFERASE_RXN_p1
0.9977071641892307
2
--------------
2-4
--------------
2-5
--------------
3-4
--------------
3-5
--------------
4-5
--------------


In [19]:

PCCdict=dict()


In [20]:
from scipy.stats import pearsonr


for rxn in totalTranscriptDict[1].keys():
    xlist = list()
    ylist = list()
    for i in range(1,6):
        xlist.append(modelDict[i].reactions.get_by_id(rxn).flux)
        ylist.append(totalTranscriptDict[i][rxn])
    ans = pearsonr(xlist,ylist)  
    PCCdict[rxn]=ans

/home/sanu/.local/lib/python2.7/site-packages/scipy/stats/stats.py:3038 RuntimeWarning: invalid value encountered in double_scalars


In [21]:
fout = open("Data/PPC_transcripts_fluxes_lightNotForced_constMaint_10_10_19.csv","w")

for rxn in PCCdict.keys():
    fout.write(rxn+","+str(PCCdict[rxn][0])+","+str(PCCdict[rxn][1])+"\n")
fout.close()

In [25]:
rxn = "SUCROSE_SYNTHASE_RXN_c1"
xlist = list()
ylist = list()
for i in range(1,6):
    xlist.append(modelDict[i].reactions.get_by_id(rxn).flux)
    ylist.append(totalTranscriptDict[i][rxn])

In [26]:
xlist

[0.014835058910007371,
 0.0168690400497365,
 0.014835058909959667,
 0.007785508992527683,
 0.0024995325175661436]

In [27]:
ylist

[16.244093349375,
 15.220867541718752,
 16.92877199385714,
 21.772953956249996,
 26.22324397875]

## References
- Kannan, K., Wang, Y., Lang, M., Challa, G. S., Long, S. P., & Marshall-Colon, A. (2019). Combining gene network, metabolic and leaf-level models shows means to future-proof soybean photosynthesis under rising CO2. In Silico Plants, 1(1), 1–18. https://doi.org/10.1093/insilicoplants/diz008
- Haile, F. J., & Higley, L. G. (2003). Changes in Soybean Gas-Exchange After Moisture Stress and Spider Mite Injury. Environmental Entomology, 32(3), 433–440. https://doi.org/10.1603/0046-225x-32.3.433